In [1]:
%matplotlib inline
%load_ext autoreload
%load_ext watermark


import dask
import distributed
import matplotlib as mpl
import matplotlib.pyplot as plt
import ncar_jobqueue
import numpy as np
import pandas as pd
import xarray as xr
import xgcm
import cartopy as cr

mpl.rcParams["savefig.dpi"] = 300
mpl.rcParams["savefig.bbox"] = "tight"
mpl.rcParams["figure.dpi"] = 180

xr.set_options(keep_attrs=True, display_style="html")

%watermark -iv

dask         : 2021.6.0
pandas       : 1.2.4
matplotlib   : 3.4.2
cartopy      : 0.19.0.post1
json         : 2.0.9
sys          : 3.9.4 | packaged by conda-forge | (default, May 10 2021, 22:13:33) 
[GCC 9.3.0]
xgcm         : 0.5.2
distributed  : 2021.6.0
numpy        : 1.20.3
ncar_jobqueue: 2021.4.14
xarray       : 0.18.2



In [3]:
cluster = ncar_jobqueue.NCARCluster(
    project="ncgd0011"
 #   scheduler_options=dict(dashboard_address=":44275"),
)

cluster.scale(4)

client = distributed.Client(cluster)
client

/glade/u/home/pmora/.conda/envs/etpac/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33104 instead
  warnings.warn(


Client Scheduler: tcp://10.12.206.54:34918 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/pmora/proxy/33104/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
path = "/glade/p/cesmdata/cseg/inputdata/ocn/jra55/v1.3_noleap/JRA.v1.3.u_10.TL319.*.nc"
ds = xr.open_mfdataset(path, parallel=True)  
ds


<xarray.Dataset>
Dimensions:    (latitude: 320, longitude: 640, time: 178120)
Coordinates:
  * time       (time) object 1958-01-01 00:00:00 ... 2018-12-31 21:00:00
  * longitude  (longitude) float64 0.0 0.5625 1.125 1.688 ... 358.3 358.9 359.4
  * latitude   (latitude) float64 -89.57 -89.01 -88.45 ... 88.45 89.01 89.57
Data variables:
    u_10       (time, latitude, longitude) float32 dask.array<chunksize=(2920, 320, 640), meta=np.ndarray>
Attributes:
    title:        u_10 -- JRA-55 based atmosphere data on TL319 grid
    Source:       Tsujino et al., 2017. JRA-55 Based Surface Dataset for Driv...
    Conventions:  CF1.0
    history:      Original file uas_input4MIPs_atmosphericState_OMIP_MRI-JRA5...
    notes:        JRA-55 TL319 data (v1.3, 2017 Oct 18) reformatted for use i...

In [5]:
ds1 = ds.where(ds['time.year'] > 1988, drop=True)
ds1

/glade/u/home/pmora/.local/lib/python3.9/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.Dataset>
Dimensions:    (latitude: 320, longitude: 640, time: 87600)
Coordinates:
  * time       (time) object 1989-01-01 00:00:00 ... 2018-12-31 21:00:00
  * longitude  (longitude) float64 0.0 0.5625 1.125 1.688 ... 358.3 358.9 359.4
  * latitude   (latitude) float64 -89.57 -89.01 -88.45 ... 88.45 89.01 89.57
Data variables:
    u_10       (time, latitude, longitude) float32 dask.array<chunksize=(2920, 320, 640), meta=np.ndarray>
Attributes:
    title:        u_10 -- JRA-55 based atmosphere data on TL319 grid
    Source:       Tsujino et al., 2017. JRA-55 Based Surface Dataset for Driv...
    Conventions:  CF1.0
    history:      Original file uas_input4MIPs_atmosphericState_OMIP_MRI-JRA5...
    notes:        JRA-55 TL319 data (v1.3, 2017 Oct 18) reformatted for use i...

In [ ]:
ds1 = xr.open_mfdataset("/glade/p/cesmdata/cseg/inputdata/ocn/jra55/v1.3_noleap/JRA.v1.3.t_10.TL319.1958.171019.nc", parallel=True)  


In [6]:
Sel_u = ds1['u_10']
U_area = Sel_u.sel(longitude=np.arange(250,290,0.5), latitude=np.arange(5,18,0.5), method='nearest')
U_area

/glade/u/home/pmora/.local/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


<xarray.DataArray 'u_10' (time: 87600, latitude: 26, longitude: 80)>
dask.array<getitem, shape=(87600, 26, 80), dtype=float32, chunksize=(2920, 26, 80), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 1989-01-01 00:00:00 ... 2018-12-31 21:00:00
  * longitude  (longitude) float64 249.8 250.3 250.9 251.4 ... 288.6 289.1 289.7
  * latitude   (latitude) float64 4.774 5.335 5.897 6.459 ... 16.57 17.13 17.69
Attributes:
    note:           10m reference height data
    standard_name:  eastward_wind
    long_name:      Eastward Near-Surface Wind Speed
    comment:        Eastward component of the near-surface wind
    units:          m s-1
    cell_methods:   area: mean time: point
    cell_measures:  area: areacella
    history:        2018-04-11T04:22:56Z altered by CMOR: Treated scalar dime...
    coordinates:    height

In [7]:
Season_U = U_area.groupby('time.season').mean('time')

In [8]:
Season_U

<xarray.DataArray 'u_10' (season: 4, latitude: 26, longitude: 80)>
dask.array<stack, shape=(4, 26, 80), dtype=float32, chunksize=(1, 26, 80), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 249.8 250.3 250.9 251.4 ... 288.6 289.1 289.7
  * latitude   (latitude) float64 4.774 5.335 5.897 6.459 ... 16.57 17.13 17.69
  * season     (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    note:           10m reference height data
    standard_name:  eastward_wind
    long_name:      Eastward Near-Surface Wind Speed
    comment:        Eastward component of the near-surface wind
    units:          m s-1
    cell_methods:   area: mean time: point
    cell_measures:  area: areacella
    history:        2018-04-11T04:22:56Z altered by CMOR: Treated scalar dime...
    coordinates:    height

In [ ]:
Season_U.plot.contourf(x='longitude',y='latitude',col='season')